In [1]:
import pct.dl.utils as ut
import plotly.graph_objects as go
import numpy as np
import time
from tensorflow import keras
from pathlib import Path
from os import path
#from keras.preprocessing.image import load_img
import IPython.display as display

ModuleNotFoundError: ignored

In [2]:
CONV_SIZE=5
IMG_ROWS, IMG_COLS = CONV_SIZE, CONV_SIZE # input image dimensions
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
model_name="SingleC2D"
model_plot_filename="C2D.png"
show_shapes=True

In [3]:
def transform_weights(wts):
    main_list=[]
    for wtss in wts:
        #print(wtss)
        sub_list=[]
        for wtsss in wtss:
            #print("wtss", wtsss[0][0])
            sub_list.append(wtsss[0][0])
            
        main_list.append(sub_list)
    return np.asarray(main_list)

In [4]:
def create_model():
    sub_image = keras.layers.Input(shape=INPUT_SHAPE, name="image")
    c2d=keras.layers.Conv2D(1, (CONV_SIZE, CONV_SIZE), strides=(2, 2), use_bias=False, name="C2D")(sub_image)
    model = keras.models.Model(inputs=sub_image, outputs=c2d, name=model_name)   
    return model

In [5]:
model=create_model()
model.summary()
#keras.utils.plot_model(model, model_plot_filename, show_shapes=show_shapes) 

NameError: ignored

In [ ]:
wts = model.get_weights()
#print(wts)
wtsz = transform_weights(wts[0])
print(wtsz)

In [ ]:
# input data
file_to_open = "grey-boxes.png" 
gimg = ut.GreyImage(file_to_open)
gimg.open()
img = gimg.getImage()
# show the image
display.display(img)



In [ ]:
left = 57
top = 97
right = left+CONV_SIZE
bottom = top+CONV_SIZE
img1 = img.crop((left, top, right, bottom)) 
img1=img1.convert('L')

In [ ]:
display.display(img1)
display.display(img1.resize((50,50)))

In [ ]:
imageArray =np.asarray(img1)/255
print(imageArray)

In [ ]:
def train(steps, epochs):
    print("ctr loss output")
    for step in range(steps+1):
        history = model.fit(inputs, outputs, epochs=epochs, verbose=False)
        loss = history.history['loss'][-1]
        out = model(inputs).numpy()[0][0][0][0]
        print('[{:3d} {:5.2f} {:4.2f}]'.format(step*epochs, loss, out), end=" ")
        wts = model.get_weights()
        wtsz = transform_weights(wts[0])
        #print(wtsz)
        spf1.data[1].z=wtsz

In [ ]:
import tensorflow as tf
reference = 9
batch_size = 500
X = tf.constant(np.array([imageArray]), dtype=tf.float32 )
Y = tf.constant(np.full((1,), reference), dtype=tf.float32 )
dataset = tf.data.Dataset.from_tensor_slices(( X , Y )) 
dataset = dataset.shuffle( 1 ).repeat( 1 ).batch( batch_size )
iterator = dataset.__iter__()
inputs , outputs = iterator.get_next()

In [ ]:
print(inputs, outputs)

In [ ]:
learning_rate=0.003
model.compile(keras.optimizers.SGD(learning_rate), loss='mse', run_eagerly=True)


In [ ]:
wts = model.get_weights()
#print(wts)
initial_wtsz = transform_weights(wts[0])
print(initial_wtsz)

In [ ]:
from plotly.subplots import make_subplots
spfig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}]])

spfig.add_trace(
    go.Surface(z=imageArray,  showscale=False),
    row=1, col=1)


spfig.add_trace(
    go.Surface(z=wtsz),
    row=1, col=2)

spfig.update_layout(
    title_text='Sub-image and Weights',
    height=600,
    width=900
)

spf1 = go.FigureWidget(spfig)
spf1

In [ ]:
print("Target output is", reference)
train(60, 2)

In [ ]:
print(imageArray)
wts = model.get_weights()
wtsz = transform_weights(wts[0])
print("Final weights")
print(wtsz)
print("sum", np.sum(wtsz))
print("Initial weights")
print(initial_wtsz)
print("sum", np.sum(initial_wtsz))